---
title: C++ concurrency in action: ch2
tags: 小书匠,thread,join,detach,Books-C++ concurrency in action,concurrency
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# C++ concurrency in action: ch2

### Launching a thread

#### Using an ordinary function to initialize

In [1]:
#include<thread>

void do_some_thing() {}

In [2]:
void do_other_thing() {}

In [3]:
std::thread my_thread(do_some_thing);

#### Using a class with function call operator

In [4]:
class background_task
{
    public:
        void operator()() const {
            do_some_thing();
            do_other_thing();
        }
};

background_task f;
std::thread my_thread(f);

In [5]:
std::thread my_thread(background_task());

input_line_12:2:23: warning: parentheses were disambiguated as a function declaration [-Wvexing-parse]
 std::thread my_thread(background_task());
                      ^~~~~~~~~~~~~~~~~~~
input_line_12:2:24: note: add a pair of parentheses to declare a variable
 std::thread my_thread(background_task());
                       ^
                       (                )


In [6]:
std::thread my_thread((background_task()));

In [7]:
std::thread my_thread { background_task()};

#### Using a lambda function

In [ ]:
std::thread my_thread([]{
    do_some_thing();
    do_other_thing();
});

### Waiting in excecptional circumstances

Here is an example of how to wait in exceptional circumstances

```cpp
struct func;
void f()
{
    std::thread t();
    try
    {
        do_something_in_current_thread();
    }
    catch(...)
    {
        t.join();
        throw;
    }
    t.join();
}
```

It is quit verbose. A better way is to use RAII <!-- {#thread_guard} -->

```cpp
class thread_guard
{
    std::thread t;
    public:
        explicit thread_guard(std::thread& t_): t(t_) {}
        ~thread_guard()
        {
            if (t.joinable())
                t.join();
        }
    thread_guard(const thread_guard&) = delete;
    thread_guard& operator=(const thread_guard&) = delete;
};

struct func;
void f()
{
    std::thread t;
    thread_guard(t);
    do_something_in_current_thread();
}
```

## Passing arguments to a thread function

In [15]:
#include <string>
#include <iostream>
#include <thread>

void f(const std::string& s) { std::cout << s << "\n";}
int main()
{
    std::thread t(f, "hello"); // ok
    t.join();
}


hello


### Using function with non-const reference

In the internal of thread, it will call the passed function using rvalues, thus may not corporate with non-const reference. 

In [52]:
%%file test.cpp
#include <string>
#include <iostream>
#include <thread>

void f(std::string& s) { std::cout << s << "\n";} // f expects non-const reference
int main()
{
    std::string s("hello world");
    std::thread t(f, s); // s is lvalue
    t.join();
}

Overwriting test.cpp


In [50]:
!g++ test.cpp -o test -std=c++11

In file included from test.cpp:3:
/Library/Developer/CommandLineTools/usr/include/c++/v1/thread:342:5: error: attempt to use a deleted function
    __invoke(_VSTD::move(_VSTD::get<1>(__t)), _VSTD::move(_VSTD::get<_Indices>(__t))...);
    ^
/Library/Developer/CommandLineTools/usr/include/c++/v1/thread:352:5: note: in instantiation of function template specialization 'std::__1::__thread_execute<std::__1::unique_ptr<std::__1::__thread_struct, std::__1::default_delete<std::__1::__thread_struct> >, void (*)(std::__1::basic_string<char> &), std::__1::basic_string<char> , 2>' requested here
    __thread_execute(*__p, _Index());
    ^
/Library/Developer/CommandLineTools/usr/include/c++/v1/thread:368:47: note: in instantiation of function template specialization 'std::__1::__thread_proxy<std::__1::tuple<std::__1::unique_ptr<std::__1::__thread_struct, std::__1::default_delete<std::__1::__thread_struct> >, void (*)(std::__1::basic_string<char> &), std::__1::basic_string<char> > >' requested here


If f expects const reference, their will be no error

In [53]:
%%file test.cpp
#include <string>
#include <iostream>
#include <thread>

void f(const std::string& s) { std::cout << s << "\n";}
int main()
{
    std::string s("hello world");
    std::thread t(f, s);
    t.join();
}

Overwriting test.cpp


In [54]:
!g++ test.cpp -o test -std=c++11 && ./test

hello world


A workaround is to use std::ref

In [55]:
%%file test.cpp
#include <string>
#include <iostream>
#include <thread>

void f(std::string& s) { std::cout << s << "\n";} // f expects non-const reference
int main()
{
    std::string s("hello world");
    std::thread t(f, std::ref(s)); // s is lvalue
    t.join();
}

Overwriting test.cpp


In [56]:
!g++ test.cpp -o test -std=c++11 && ./test

hello world


## Transferring ownership of a thread

`std::thread` is movable but not copyable, thus it can not be copied. However, we can transferring ownership of a thread of execution among std::thread.


### transferring ownership into function and out of function

The move support in std::thread means that ownership can readily be transferred out of a function by returning a std::thread, and can also be transferred into a function by passing arguments by std::thread.

Here are examples of transferring ownership out of function.

```cpp
std::thread f()
{
    void some_function();
    return std::thread(some_function); // move constructor
}

std::thread g()
{
    void some_other_function();
    std::thread t(some_function);
    return t; // move constructor
}
```

Here are examples of transferring ownership into function.

```cpp
void f(std::thread t);
void g()
{
    void some_function();
    f(std::thread()); // move constructor
    std::thread t(some_function);
    f(std::move(t)); // move constructor
}
```

### Scoped thread using move

In [74]:
%%file test.cpp
#include <iostream>
#include <thread>
#include <algorithm>

struct func
{
    int& i;
    func(int& i_): i(i_) { }
    void operator()() const
    {
            std::cout << "i = " << i << std::endl;
    }
};

class scoped_thread
{
    std::thread t;
    public:
        explicit scoped_thread(std::thread t_): t(std::move(t_))
        {
            if (!t.joinable()) throw std::logic_error("No thread");
        }
        ~scoped_thread(){ t.join(); }
        scoped_thread(const scoped_thread& ) = delete;
        scoped_thread& operator=(const scoped_thread&) = delete;
};

void do_something_in_current_thread()
{
    std::cout << "do_something_in_current_thread" << std::endl;
}

int main()
{
    int some_local_state = 3;
    scoped_thread t {std::thread(func(some_local_state))};
    do_something_in_current_thread();
    return 0;
}

Overwriting test.cpp


In [75]:
!g++ -std=c++11 test.cpp -o test && ./test

do_something_in_current_thread
i = 3


Note that `scoped_thread` here is different from [thread_guard](#thread_guard) in the following aspects

1. scoped_thread's constructor expects a `std::thread` as an argument, while `thread_guard` expects `std::thread&` as an argument. Because std::thread supports move, using a `std::thread` as a parameter may invoke move instead of copy.
Thus, scoped_thread will take over the ownership of the passed `std::thread` and no objects else could manipulate the thread. While `thread_guard` merely references a thread object, there will be other objects manipulating the thread.

2. `scoped_thread` checks whether t is joinable in the constructor instead of the destructor, while `thread_guard` is the opposite.

Because `scoped_thread` takes the ownership of `std::thread`, once the joinability of the thread is guaranteed, there is no need to be checked again.

Whereas `thread_guard` does not need to check `t.joinable()` in the constructor but does in the destructor, because there may be other objects join or detach the thread. It cannot ensure the joinability of the thread, thus the joinability should be checked every time before joining.

### Using move-aware container with std::thread

In [87]:
%%file test.cpp
#include <iostream>
#include <thread>
#include <vector>

void do_work(int i)
{
        std::cout << i << ": hello world" << std::endl;
}

int main()
{
    std::vector<std::thread> threads;
    for(unsigned i=0; i<5; ++i)
    {
        threads.emplace_back(do_work, i); // spawns threads
    }
    for (auto& entry: threads)
        entry.join(); // join
    return 0;
}

Overwriting test.cpp


In [88]:
!g++ test.cpp -o test -std=c++11 && ./test

0: hello world
3: hello world
2: hello world
41: hello world: hello world



# Reference